In [ ]:
import torch
from torch import nn
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.transforms  import ToTensor
import torch.nn.functional as F

In [ ]:
#Dowloading the training and testing datasets
training_set = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)
test_set = datasets.FashionMNIST(
    root = "data",
    train = False,
    download= True,
    transform =  ToTensor(),
)

In [ ]:
batch_size = 64  #Batches of 64
training_loader = DataLoader(training_set,batch_size=batch_size)
test_loader = DataLoader(test_set,batch_size=batch_size)


In [ ]:
class MPS(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()  #changing the tensor in 1D
    self.linear_relu_stack = nn.Sequential(
      nn.Linear(28*28,64),
      nn.ReLU(), #Activation function for first hidden layer
      nn.Linear(64,128),
      nn.ReLU(),  #For second hidden layer
      nn.Linear(128,10),
      # nn.Softmax(dim=1)
    )
  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = MPS()
print(model)


MPS(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [ ]:
loss_function = nn.CrossEntropyLoss()   #Here the softmax function is already incorperated
                                        #this cross entropy loss function
optimizer = torch.optim.SGD(model.parameters(),lr = 0.1) #Learning rate changed to 0.1


In [ ]:
def train(dataloader,model,loss_function,optimizer):
  size = len(dataloader.dataset)
  model.train() #setting the model in training mode
  for batch,(X,y) in enumerate(dataloader):
    optimizer.zero_grad()
    pred = model(X)  #model's prediction
    loss = loss_function(pred,y) #loss between predicted o/p and ground truth
    loss.backward()  #backpropagation
    optimizer.step()
    if batch%100 == 0:
      loss,current = loss.item(),(batch+1)*len(X)
      print(f"Loss : {loss:>7f} [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader,model,loss_function):
  size = len(dataloader.dataset) #total size
  no_of_batches = len(dataloader)
  model.eval()  #setting in evaluation mode
  test_loss , correct = 0,0
  with torch.no_grad(): #no gradient computation during testing
    for X,y in dataloader:
      pred = model(X)
      test_loss += loss_function(pred,y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= no_of_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 10
for t in range(epochs):
  print(f"Epoch {t+1}\n")
  train(training_loader,model,loss_function,optimizer)
  test(test_loader,model,loss_function)
print("ALl DONE!!!")

Epoch 1

Loss : 0.202866 [   64/60000]
Loss : 0.309743 [ 6464/60000]
Loss : 0.178530 [12864/60000]
Loss : 0.256628 [19264/60000]
Loss : 0.264839 [25664/60000]
Loss : 0.328915 [32064/60000]
Loss : 0.273957 [38464/60000]
Loss : 0.337277 [44864/60000]
Loss : 0.317218 [51264/60000]
Loss : 0.345660 [57664/60000]
Test Error: 
 Accuracy: 87.4%, Avg loss: 0.350244 

Epoch 2

Loss : 0.202768 [   64/60000]
Loss : 0.304116 [ 6464/60000]
Loss : 0.177279 [12864/60000]
Loss : 0.244934 [19264/60000]
Loss : 0.269842 [25664/60000]
Loss : 0.321473 [32064/60000]
Loss : 0.271453 [38464/60000]
Loss : 0.339924 [44864/60000]
Loss : 0.312394 [51264/60000]
Loss : 0.337895 [57664/60000]
Test Error: 
 Accuracy: 87.2%, Avg loss: 0.354012 

Epoch 3

Loss : 0.190738 [   64/60000]
Loss : 0.278504 [ 6464/60000]
Loss : 0.175181 [12864/60000]
Loss : 0.242947 [19264/60000]
Loss : 0.250175 [25664/60000]
Loss : 0.310642 [32064/60000]
Loss : 0.269113 [38464/60000]
Loss : 0.318400 [44864/60000]
Loss : 0.300927 [51264/60000]